In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [6]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [7]:
batch_size = 16
learning_rate = 0.0002
num_epoch = 10

In [8]:
mnist_train = dset.MNIST("/content/drive/MyDrive", train=True, transform=transforms.ToTensor(), target_transform=None, download=False)
mnist_test = dset.MNIST("/content/drive/MyDrive", train=False, transform=transforms.ToTensor(), target_transform=None, download=False)

In [9]:
mnist_train.__getitem__(0)[0].size(), mnist_train.__len__()

(torch.Size([1, 28, 28]), 60000)

In [10]:
mnist_test.__getitem__(0)[0].size(), mnist_test.__len__()

(torch.Size([1, 28, 28]), 10000)

In [11]:
mnist_test.__getitem__(5888)[1]

4

In [12]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

In [13]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()   # parents의 __init__()을 사용하기 위함!
    
    self.layer = nn.Sequential(
        nn.Conv2d(1,16,5),
        nn.ReLU(),
        nn.Conv2d(16,32,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2),
        nn.Conv2d(32,64,5),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )

    self.fc_layer = nn.Sequential(
        nn.Linear(64*3*3, 100),
        nn.ReLU(),
        nn.Linear(100,10)
    )

  def forward(self, x):
    out = self.layer(x)
    out = out.view(batch_size, -1)
    out = self.fc_layer(out)
    return out

model = CNN().cuda()

In [14]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
for i in range(num_epoch):
  for j,[image,label] in enumerate(train_loader):
    x = image.cuda()
    y_ = label.cuda()

    optimizer.zero_grad()
    output = model.forward(x)
    loss = loss_func(output, y_)
    loss.backward()
    optimizer.step()

    if j % 1000 == 0:
      print(loss)

tensor(2.2922, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3095, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3035, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3170, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2983, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2797, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3018, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2934, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3020, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2948, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3030, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2813, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2887, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2842, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3068, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2729, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2648, device='cuda:0', grad_fn=<NllLossBackward

In [16]:
correct = 0
total = 0

for image,label in test_loader:
  x = Variable(image, volatile=True).cuda()
  y_ = label.cuda()

  output = model.forward(x)
  _, output_index = torch.max(output, 1)

  total += label.size(0)
  correct += (output_index == y_).sum().float()

print("Accuracy of Test Data: {}".format(100*correct/total))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  """


Accuracy of Test Data: 89.07999420166016


In [17]:
for image,label in test_loader:
  x = image.cuda()
  y = label.cuda()

  output = model.forward(x)
  _, output_index = torch.max(output,1)

  print(x.size())
  print(y.size())
  print(output.size()) 
  print(torch.max(output,1)) 
  print(output_index)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
values=tensor([7.1531, 7.2189, 7.6683, 5.0823, 7.1439, 6.7419, 7.8574, 9.7813, 7.2808,
        6.5201, 4.6797, 4.1755, 7.5960, 4.3610, 4.5281, 3.4436],
       device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([7, 9, 6, 3, 6, 3, 7, 4, 6, 5, 3, 5, 4, 7, 8, 2], device='cuda:0'))
tensor([7, 9, 6, 3, 6, 3, 7, 4, 6, 5, 3, 5, 4, 7, 8, 2], device='cuda:0')
torch.Size([16, 1, 28, 28])
torch.Size([16])
torch.Size([16, 10])
torch.return_types.max(
values=tensor([4.4678, 7.6556, 4.9339, 7.2040, 2.6622, 4.0005, 8.9367, 2.5884, 3.7846,
        8.1951, 6.7780, 5.8491, 6.7794, 8.1877, 8.5977, 3.6177],
       device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([8, 0, 7, 6, 8, 9, 7, 8, 3, 1, 9, 5, 2, 7, 3, 1], device='cuda:0'))
tensor([8, 0, 7, 6, 8, 9, 7, 8, 3, 1, 9, 5, 2, 7, 3, 1], device='cuda:0')
torch.Size([16, 1, 28, 28])
torch.Size([16])
torch.Size([16, 10])
torch.return_types.max(
values=tensor([ 8.0128,  5.6719,  3.7147,  8.7988,  9.1207,  8.7066,